In [1]:
import os
import datetime
import pandas as pd
import numpy as np
from collections import Counter

import warnings
warnings.filterwarnings("ignore")

In [2]:
df=pd.read_csv('data2/dataset.csv',names=['Date','Time','Sensor','State','Activity','Note'])

### Data PreProcess-Change Outliers

In [3]:
df.replace({'22009-02-03':'03/02/2009'},inplace=True)

In [4]:
df['Datetime']=df['Date'].map(lambda x: str(x)+' ')+df['Time']

In [5]:
df.replace({'04/03/2009 09:017:41.696759':'04/03/2009 09:17:42'},inplace=True)

In [6]:
df=df[~df.T.isnull().all()==True]

In [7]:
df['Datetime']=[datetime.datetime.strptime(str(time), '%d/%m/%Y %H:%M:%S').strftime('%Y-%m-%d %H:%M:%S') for time in df['Datetime']]

In [8]:
df['Datetime']=pd.to_datetime(df['Datetime'])

In [9]:
df=df.drop(['Date','Time'],axis=1)

In [10]:
df=df[['Datetime','Sensor','State','Activity','Note']]

In [11]:
df = df.sort_values(by = "Datetime",ascending=True)

In [12]:
df = df.reset_index(drop=True)

In [13]:
R1_Bed_to_Toilet_begin=df[df['Activity'].str.contains('R1_Bed_to_Toilet_begin')==True]

In [14]:
R1_Bed_to_Toilet_end=df[df['Activity'].str.contains('R1_Bed_to_Toilet_end')==True]

In [15]:
R1_Bed_to_Toilet_begin.loc[:,'Activity']=R1_Bed_to_Toilet_begin.loc[:,'Activity'].apply(lambda x: 'R1_Bed_to_Toilet')
R1_Bed_to_Toilet_begin.loc[:,'Note']=R1_Bed_to_Toilet_begin.loc[:,'Note'].apply(lambda x: 'begin')
R1_Bed_to_Toilet_end.loc[:,'Activity']=R1_Bed_to_Toilet_end.loc[:,'Activity'].apply(lambda x: 'R1_Bed_to_Toilet')
R1_Bed_to_Toilet_end.loc[:,'Note']=R1_Bed_to_Toilet_end.loc[:,'Note'].apply(lambda x: 'end')

In [16]:
remain=df[df['Activity'].str.contains('R1_Bed_to_Toilet_begin')!=True]

In [17]:
remain=remain[remain['Activity'].str.contains('R1_Bed_to_Toilet_end')!=True]

In [18]:
table=remain.append(R1_Bed_to_Toilet_begin).append(R1_Bed_to_Toilet_end).sort_values(by = "Datetime",ascending=True)

In [19]:
BE=table[~table['Note'].isnull()==True]

In [20]:
table.drop([131693,134839,127365,126404],inplace=True)

In [21]:
Table1=table.copy()

In [22]:
group = table.groupby('Activity')
#group.describe()

In [23]:
groups = {}
for k, g in group:
    groups[k]=g  
for i in groups:
    groups[i]=pd.DataFrame(groups[i])

In [24]:
R1_work=groups['R1_Work']

In [25]:
ab=[4,6,8,10,11,16,17,19]

In [26]:
change=[R1_work.index[t] for t in ab]

In [27]:
for t in change:
    Table1.loc[t,'Activity':'Note']=np.nan 

In [28]:
date=['2009-02-03 17:57:52','2009-02-04 21:05:59','2009-02-05 18:38:52','2009-02-06 14:29:30','2009-02-06 14:56:15','2009-02-09 18:53:58','2009-02-09 19:30:53','2009-02-10 11:08:15']

In [29]:
Ch=[]

In [30]:
for d in date:
    Change=Table1[Table1['Datetime']==d]
    for r in range(Change.shape[0]):
        Ch.append(Change.iloc[r,:])

In [31]:
Ch=pd.DataFrame(Ch)

In [32]:
Ch=Ch[Ch['Sensor'].str.contains('M31')==True]

In [33]:
Chindex=Ch.index.to_list()

In [34]:
be=Chindex[0:4]

In [35]:
be.append(Chindex[5])

In [36]:
en=[14208, 20990, 22671]

In [37]:
for t in be:
    Table1.loc[t,'Activity']='R1_Work' 
    Table1.loc[t,'Note']='begin' 

In [38]:
for t in en:
    Table1.loc[t,'Activity']='R1_Work' 
    Table1.loc[t,'Note']='end' 

In [39]:
R2_work=groups['R2_Work']

In [40]:
ab2=[0,2,4,6,7,35]

In [41]:
change=[R2_work.index[t] for t in ab2]

In [42]:
for t in change:
    Table1.loc[t,'Activity':'Note']=np.nan 

In [43]:
date=['2009-02-02 15:23:24','2009-02-04 20:18:07','2009-02-05 20:03:56','2009-02-09 10:54:37','2009-02-09 11:31:40','2009-02-26 17:17:45']

In [44]:
Ch=[]

In [45]:
for d in date:
    Change=Table1[Table1['Datetime']==d]
    for r in range(Change.shape[0]):
        Ch.append(Change.iloc[r,:])

In [46]:
Ch=pd.DataFrame(Ch)

In [47]:
Ch1=Ch[Ch['Sensor'].str.contains('M44')==True]

In [48]:
Ch2=Ch[Ch['Sensor'].str.contains('M45')==True]

In [49]:
Ch=Ch1.append(Ch2)

In [50]:
Ch=Ch.sort_values(by = "Datetime",ascending=True)

In [51]:
Ch=Ch.drop([19521],axis=0)

In [52]:
Chindex=Ch.index.to_list()

In [53]:
be=Chindex[0:4]

In [54]:
en=Chindex[4:6]

In [55]:
for t in be:
    Table1.loc[t,'Activity']='R2_Work' 
    Table1.loc[t,'Note']='begin' 

In [56]:
for t in en:
    Table1.loc[t,'Activity']='R2_Work' 
    Table1.loc[t,'Note']='end' 

In [57]:
Table1['Note'].value_counts()

begin    514
end      514
Name: Note, dtype: int64

### Data Transform

In [58]:
beginandend=Table1[~Table1['Note'].isnull()==True]

In [59]:
gro = beginandend.groupby('Activity')
gros = {}
for k, g in gro:
    gros[k]=g  
for i in gros:
    gros[i]=pd.DataFrame(gros[i])

In [60]:
for k, g in gros.items():
    print(k)

Clean
Meal_Preparation
R1_Bed_to_Toilet
R1_Personal_Hygiene
R1_Sleep
R1_Work
R2_Bed_to_Toilet
R2_Personal_Hygiene
R2_Sleep
R2_Work
Study
Wash_Bathtub
Watch_TV


In [61]:
Term=list()

In [62]:
for item in gros:
    gr = gros[item].groupby('Note')
    grs = {}
    for k, g in gr:
        grs[k]=g  
    for i in grs:
        grs[i]=pd.DataFrame(grs[i])
    term=zip(grs['begin'].index.to_list(),grs['end'].index.to_list())
    Term1=list(term)
    Term+=Term1
    

In [63]:
Term=sorted(Term)

In [64]:
Table=Table1.copy()

In [65]:
for m,n in list(term):
    Table.loc[m:n,'Activity']=Table1.loc[m,'Activity']

In [66]:
Table=Table.reset_index(drop=True)

In [67]:
Sensor=Table['Sensor'].value_counts()

In [68]:
sensor=Sensor.index.tolist()

In [69]:
gp = table.groupby('Activity')

In [70]:
gps = {}
for k, g in gp:
    gps[k]=g  
for i in gps:
    gps[i]=pd.DataFrame(gps[i])

In [71]:
Table['Duration']=0
Table['BeginTime']=0

In [72]:
for b,e in Term:
    Table['Activity'][b:e+1]=Table1['Activity'][b] 
    Table['Duration'][b:e+1]=Table1['Datetime'][e]-Table['Datetime'][b]
    Table['BeginTime'][b:e+1]=Table1['Datetime'][b]

In [73]:
new=Table[['BeginTime','Duration','Sensor','Activity']]

In [74]:
sensor.insert(0,'BeginTime')
sensor.insert(1,'Duration')
sensor.insert(73,'Activity')

In [75]:
pred=pd.DataFrame(columns=sensor)

In [76]:
for b,e in Term:
    dic={sen:0 for sen in sensor}
    dic=pd.DataFrame([dic])
    series=Table['Sensor'][b:e+1].value_counts().to_dict()
    dic['BeginTime']=Table['BeginTime'][b]
    dic['Duration']=Table['Duration'][b]
    dic['Activity']=Table['Activity'][b]
    for k,v in series.items():
        dic[k] = v
    pred = pred.append(dic)
    

In [77]:
series=Table['Sensor'][0:80].value_counts().to_dict()

In [78]:
pred=pred.reset_index(drop=True)

In [79]:
pred['Duration']=[datetime.datetime.strptime(str(time), '0 days %H:%M:%S').strftime('%H:%M:%S') for time in pred['Duration']]

In [80]:
pred['BeginTime']=[datetime.datetime.strptime(str(time), '%Y-%m-%d %H:%M:%S').strftime('%H:%M:%S') for time in pred['BeginTime']]

In [82]:
pred.to_csv('./data2/table.csv',index=False)

### Up&downstair

In [83]:
updown=pd.read_csv('data2/stairsData.csv')

In [84]:
updown1=updown.drop_duplicates(subset = ['start'])
updown1=updown1.drop_duplicates(subset = ['end'])

In [85]:
updown1.reset_index(inplace=True,drop=True)

In [86]:
ups=pd.DataFrame([])
for i in range(len(updown1.start)):
    ups=ups.append(Table1[Table1['Datetime'].where(Table1['Datetime']==updown1.start[i]).notnull()==True]) 

In [87]:
ups= ups.drop_duplicates(subset = ['Datetime'])

In [88]:
upe=pd.DataFrame([])
for i in range(len(updown1.end)):
    upe=upe.append(Table1[Table1['Datetime'].where(Table1['Datetime']==updown1.end[i]).notnull()==True]) 

In [89]:
upe= upe.drop_duplicates(subset = ['Datetime'])

In [90]:
udt=zip(ups.index.to_list(),upe.index.to_list())

In [91]:
udt=list(udt)

In [92]:
udpred=pd.DataFrame(columns=sensor)

In [93]:
for b,e in udt:
    dic={sen:0 for sen in sensor}
    dic=pd.DataFrame([dic])
    series=Table['Sensor'][b:e+1].value_counts().to_dict()
    dic['BeginTime']=Table['BeginTime'][b]
    dic['Activity']='up&downstairs'
    for k,v in series.items():
        dic[k] = v
    udpred = udpred.append(dic)

In [94]:
udpred.reset_index(drop=True,inplace=True)

In [95]:
udpred.Duration=updown1.duration

In [96]:
udpred['Duration']=[datetime.datetime.strptime(str(time), '0 days %H:%M:%S').strftime('%H:%M:%S') for time in udpred['Duration']]
udpred['BeginTime']=[datetime.datetime.strptime(str(time), '%Y-%m-%d %H:%M:%S').strftime('%H:%M:%S') for time in udpred['BeginTime']]

In [97]:
Final=pd.concat([pred,udpred]).reset_index(drop=True)

In [98]:
Final.to_csv('./data2/table_add_ud.csv',index=False)